In [1]:
cd "/content/drive/My Drive/audioToLyrics"

/content/drive/My Drive/audioToLyrics


In [2]:
from audiotolyrics.preprocessing.datacontainer import audioContainer, textContainer
from audiotolyrics.textgenmodel.atlmodel import  audioFeatureExtractor

In [3]:
paths_audio_text = []
for genre in ["rap", "country"]:
  paths_audio_text.append(f"./audioToLyrics_{genre}")

In [28]:
songs = audioContainer(paths_audio_text, sr = 2000, limit=None)

Number of files in folder: 290
Number of audio tracks with lyrics: 11
Number of files in folder: 157
Number of audio tracks with lyrics: 38


In [29]:
songs.paths

['./audioToLyrics_rap/Sublime - Date Rape-CeMeDihwyrg.mp3',
 './audioToLyrics_rap/Busta Rhymes, Rapsody - Best I Can (Audio)-B-vxUg51-jM.mp3',
 './audioToLyrics_rap/Rapsody - 12 Problems (Extended Version)-_mkhkaw84k8.mp3',
 "./audioToLyrics_rap/2Pac Ft Rappin' 4-Tay Only God Can Judge Me-JNbpBLvZsEw.mp3",
 './audioToLyrics_rap/Afroman - Colt 45-rNtiEcT9W6M.mp3',
 "./audioToLyrics_rap/$UICIDEBOY$ - You're Now Tuning in to 66.6 FM with DJ Rapture (The Hottest Hour of the Evening)-CYlADTZf-7Y.mp3",
 './audioToLyrics_rap/Chance The Rapper ft. Saba - Angels-eedeXTWZUn8.mp3',
 './audioToLyrics_rap/The Worst Guys (Feat. Chance The Rapper) - Childish Gambino-dwUug-ucBLc.mp3',
 "./audioToLyrics_rap/KRS-One - Step Into A World (Rapture's Delight)-xbJxcFyaCpI.mp3",
 './audioToLyrics_rap/Anita Baker - Caught Up in the Rapture-wYYDaDPmqBs.mp3',
 './audioToLyrics_rap/Vin Jay - Mumble Rapper vs Lyricist-1lmpGxQnjqk.mp3',
 './audioToLyrics_country/Travis Tritt - Country Club (Country Club)-QRqHA5Q_5v

In [30]:
songs.load_songs()

In [31]:
text = textContainer(paths_audio_text, songs.audio_with_text, songs.audiowaves)

In [ ]:
input_audio, input_text, output_text = text.run_datapipeline()

In [9]:
input_audio.shape, input_text.shape, output_text.shape

((2562, 1, 30000), (2562, 15), (2562, 592))

In [10]:
text.end_index_songs

[687, 895, 394, 372, 214]

In [11]:
# Setting some hyperparameters
ks_list = [9, 3, 3]
filter_list = [16, 32, 64]
stride_list = [4,4,4]
laten_dim = 256
embedding_dim = 128

In [16]:
model = audioFeatureExtractor(text.vocab_length, filter_list, ks_list, stride_list, embedding_size=embedding_dim,
                              latent_dim=laten_dim, max_len= text.max_len,
                             input_size=1,seq_len = input_audio.shape[-1])

In [17]:
model.traintextgen(input_audio, input_text, output_text, epochs=50,)

Starting model training for 50 epochs


/content/drive/My Drive/audioToLyrics/audiotolyrics/textgenmodel/atlmodel.py:81: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  text, audio = torch.tensor(text).type(torch.LongTensor), torch.tensor(audio)


Epoch #2, loss=5.002322413303234
Epoch #3, loss=4.76282611305331
Epoch #4, loss=4.707406978548309
Epoch #5, loss=4.509986871554528
Epoch #6, loss=4.3984636112495705
Epoch #7, loss=3.97965219432925
Epoch #8, loss=3.877811077936196
Epoch #9, loss=3.791047383237768
Epoch #10, loss=3.6077907471745103
Epoch #12, loss=3.2676179193788104
Epoch #13, loss=3.200704699865094
Epoch #14, loss=3.017869522726094
Epoch #15, loss=2.823215022775126
Epoch #16, loss=2.546127681792886
Epoch #17, loss=2.4196510583530237
Epoch #18, loss=2.4105405044040564
Epoch #19, loss=2.3205603877036474
Epoch #20, loss=2.205055143960096
Epoch #22, loss=1.812074201060804
Epoch #23, loss=1.7222068625815603
Epoch #24, loss=1.557256889089735
Epoch #25, loss=1.5247430321347273
Epoch #26, loss=1.3660338796376261
Epoch #27, loss=1.2118631057242897
Epoch #28, loss=1.1096253968260745
Epoch #29, loss=1.0613023158212058
Epoch #30, loss=1.1117437334183637
Epoch #32, loss=0.9526794554563641
Epoch #33, loss=0.7830761370800987
Epoch #34

In [26]:
model.spitbars(text.tokenizer, input_audio[6], in_text="street")

'street know and go blacker two pics two see shes my start start start start start'

In [27]:
model.spitbars(text.tokenizer, input_audio[-1], in_text="street")

'street start start til til and orleans up minute minute now start my ride start start'